In [516]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [517]:
dados = pd.read_excel('dados_relatorio.xlsx')

In [518]:
dados = dados.iloc[1000:,:]

In [519]:
rsi_period = 3
l_name = []
for i in dados:
    if i != 'time':
        a = str(i).split()[1]
        chg = dados[i].diff(1) #variação em 1 vela
        gain = chg.mask(chg<0,0) #pega somente as variacoes positivas
        loss = chg.mask(chg>0,0) #pega somente as variacoes negativas
        #dados['gain'] = gain
        #dados['loss'] = loss
        avg_gain = gain.ewm(com = rsi_period - 1, min_periods = rsi_period).mean() #exponential weighted function
        avg_loss = loss.ewm(com = rsi_period - 1, min_periods = rsi_period).mean()
        rs = abs(avg_gain/avg_loss)
        rsi = 100-(100/(1+rs))
        dados['RSI '+a] = rsi

In [520]:
dados

,time,close IBOV,close ABEV3,close AZUL4,close B3SA3,close BBAS3,close BBDC3,close BBDC4,close BBSE3,close BEEF3,...,RSI TAEE11,RSI TIMP3,RSI TOTS3,RSI UGPA3,RSI USIM5,RSI VALE3,RSI VIVT4,RSI VVAR3,RSI WEGE3,RSI YDUQ3
1000,2016-01-21,37717,14.60,NaN,9.00,10.72,8.94,8.20,14.96,11.18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001,2016-01-22,38031,15.08,NaN,8.97,10.96,8.97,8.37,15.14,11.53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002,2016-01-26,37497,15.07,NaN,8.89,10.87,8.97,8.28,14.47,11.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1003,2016-01-27,38376,15.94,NaN,9.17,10.71,9.15,8.38,14.51,11.38,...,100.000000,73.134328,100.000000,68.324125,10.000000,81.115880,81.489842,NaN,65.333333,21.951220
1004,2016-01-28,38630,16.01,NaN,9.26,10.95,9.31,8.39,14.98,11.58,...,100.000000,41.263158,100.000000,73.019608,5.970149,67.439786,57.120253,NaN,61.635220,14.173228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970,2019-12-20,115121,18.91,57.00,47.96,50.45,30.09,32.23,34.54,12.08,...,83.326686,87.827148,79.154630,98.909230,78.991718,93.732797,93.733443,79.412482,63.208688,55.237650
1971,2019-12-23,115863,18.96,57.60,45.78,51.98,30.70,32.43,34.73,12.07,...,91.441497,89.786431,43.057640,99.400909,92.175716,77.864041,96.295438,88.316961,57.770578,65.684260
1972,2019-12-26,117203,19.22,57.07,45.57,52.57,31.02,32.83,34.56,12.64,...,96.289222,92.902159,30.301967,99.523444,92.175716,82.346970,97.703671,90.880726,87.039821,56.685335
1973,2019-12-27,116534,19.16,57.80,43.19,52.34,30.82,32.71,34.63,12.74,...,80.246791,92.902159,10.101789,99.580628,54.030087,82.598689,67.204124,51.648178,91.273609,60.814923


In [574]:
#cond 
# or (preco_acao - preco_compra)*volume <= -.02*carteira)
def opera(dados_rsiticker, gain):
    print('\n',_ticker)
    dic[_ticker] = {}
    dic[_ticker]['Qnt Trades'] = 0
    dic[_ticker]['Resultado'] = 0
    dic[_ticker]['Periodo Carregando'] = []
    dic[_ticker]['Carteira'] = 0
    
    posicao = 0 #vendido/neutro/comprado
    resultado = 0 
    preco_compra = 0
    carteira = 3000
    operacao = 0 #carteira provisória pra evitar bug
    dias_carregando = 0
    volume = 0
    for i in dados_rsiticker.index:
        dia = str(dados['time'][dados.index == i].values[0])[:10]
        rsi = dados_rsiticker[dados_rsiticker.index == i].values[0]
        preco_acao = dados[ticker][dados[ticker].index == i].values[0]
        
        if posicao == 1:
            duracao = i-dias_carregando

        #comprar
        if rsi <= 30 and posicao == 0 and carteira >= 0:
            posicao = 1 #agora está comprado em 1
            preco_compra = preco_acao #salva o preço de compra
            volume = (carteira//preco_acao)
            operacao -= volume*preco_acao #desconta na carteira o valor da ação
            dias_carregando = i
        #zerar
        elif posicao == 1 and preco_acao - preco_compra >= gain*preco_compra:
            #se tiver comprado e lucro de x%
            operacao += preco_acao*volume #credita o valor da ação vendida
            posicao = 0
            dic[_ticker]['Qnt Trades'] += 1
            carteira += operacao
            dic[_ticker]['Periodo Carregando'].append(duracao)
            print(f'{dia} - Saldo: {carteira:.2f} Gain:  {operacao:.2f} | Compra : {preco_compra:.2f} -Venda: {preco_acao:.2f} | {duracao} dias')
            if operacao >= 0:
                trade_positivo.append(operacao)
            else:
                trade_negativo.append(operacao)
            operacao = 0
            dias_carregando = 0
        #encerra posicao baseado no tempo ou stop loss
        elif posicao == 1 and (duracao >= 40 or (preco_acao - preco_compra)*volume <= -.05*carteira and rsi>=60):
            #se tiver comprado e deu o limite de tempo ou stop financeiro
            operacao += preco_acao*volume #credita o valor da ação vendida -> força a venda 
            posicao = 0
            dic[_ticker]['Qnt Trades'] += 1
            carteira += operacao
            dic[_ticker]['Periodo Carregando'].append(duracao)
            if operacao >= 0:
                trade_positivo.append(operacao)
                print(f'{dia} - Saldo: {carteira:.2f} Stop:  {operacao:.2f} | Compra : {preco_compra:.2f} -Venda: {preco_acao:.2f} | {duracao} dias')

            else:
                trade_negativo.append(operacao)
                print(f'{dia} - Saldo: {carteira:.2f} Stop: {operacao:.2f} | Compra : {preco_compra:.2f} -Venda: {preco_acao:.2f} | {duracao} dias')
            dias_carregando = 0
            operacao = 0      
            
    if carteira == carteira: #prevente contra NaN values 
        dic[_ticker]['Resultado'] = carteira
    return dic

In [575]:
doc = {}
trade_positivo = []
trade_negativo = []
l = [.08]
for pct in l:
    valor_carteira = 0
    dic = {}
    for ticker in dados.iloc[:,2:76:1]:
        _ticker = (ticker.split()[1])
        rsi_ticker = ('RSI '+_ticker)
        cotacoes = dados[rsi_ticker]        
        opera(cotacoes,pct) #executa função para compra e venda para cada acao
        doc[str(pct)] = dic


 ABEV3
2016-03-07 - Saldo: 3246.28 Gain:  246.28 | Compra : 15.95 -Venda: 17.26 7 dias
2016-05-10 - Saldo: 3399.94 Stop:  153.66 | Compra : 16.41 -Venda: 17.19 40 dias
2016-07-13 - Saldo: 3523.77 Stop:  123.83 | Compra : 16.71 -Venda: 17.32 40 dias
2016-09-12 - Saldo: 3671.37 Stop:  147.60 | Compra : 17.18 -Venda: 17.90 40 dias
2016-11-10 - Saldo: 3440.49 Stop: -230.88 | Compra : 17.61 -Venda: 16.50 40 dias
2016-12-26 - Saldo: 3227.64 Stop: -212.85 | Compra : 15.96 -Venda: 14.97 30 dias
2017-03-29 - Saldo: 3458.16 Stop:  230.52 | Compra : 15.76 -Venda: 16.89 40 dias
2017-05-04 - Saldo: 3734.57 Gain:  276.41 | Compra : 16.37 -Venda: 17.68 14 dias
2017-07-14 - Saldo: 3758.00 Stop:  23.43 | Compra : 17.46 -Venda: 17.57 40 dias
2017-09-15 - Saldo: 4079.18 Gain:  321.18 | Compra : 18.52 -Venda: 20.11 9 dias
2017-11-27 - Saldo: 3995.13 Stop: -84.05 | Compra : 19.85 -Venda: 19.44 40 dias
2018-01-11 - Saldo: 4323.77 Gain:  328.64 | Compra : 19.16 -Venda: 20.74 26 dias
2018-03-12 - Saldo: 4760

2019-02-14 - Saldo: 12506.04 Gain:  1034.56 | Compra : 46.94 -Venda: 51.18 3 dias
2019-04-05 - Saldo: 11863.44 Stop: -642.60 | Compra : 48.99 -Venda: 46.47 25 dias
2019-05-08 - Saldo: 12841.64 Gain:  978.20 | Compra : 44.22 -Venda: 47.87 16 dias
2019-05-30 - Saldo: 13888.84 Gain:  1047.20 | Compra : 45.77 -Venda: 49.51 13 dias
2019-07-04 - Saldo: 15165.99 Gain:  1277.15 | Compra : 48.28 -Venda: 52.73 13 dias
2019-08-21 - Saldo: 13500.17 Stop: -1665.82 | Compra : 50.84 -Venda: 45.25 29 dias
2019-09-06 - Saldo: 15021.62 Gain:  1521.45 | Compra : 42.75 -Venda: 47.58 8 dias
2019-11-11 - Saldo: 15210.86 Stop:  189.24 | Compra : 45.13 -Venda: 45.70 40 dias
2019-12-18 - Saldo: 16840.46 Gain:  1629.60 | Compra : 45.19 -Venda: 50.04 24 dias

 BBDC3
2016-03-17 - Saldo: 3363.86 Gain:  363.86 | Compra : 13.26 -Venda: 14.87 1 dias
2016-04-12 - Saldo: 3706.58 Gain:  342.72 | Compra : 14.13 -Venda: 15.57 4 dias
2016-06-03 - Saldo: 3373.38 Stop: -333.20 | Compra : 15.10 -Venda: 13.74 30 dias
2016-06-3

2017-02-07 - Saldo: 3481.60 Stop: -103.70 | Compra : 11.72 -Venda: 11.38 40 dias
2017-03-27 - Saldo: 3024.10 Stop: -457.50 | Compra : 11.38 -Venda: 9.88 31 dias
2017-05-03 - Saldo: 3296.90 Gain:  272.80 | Compra : 9.75 -Venda: 10.63 7 dias
2017-06-05 - Saldo: 3569.52 Gain:  272.62 | Compra : 10.40 -Venda: 11.26 16 dias
2017-07-10 - Saldo: 3860.22 Gain:  290.70 | Compra : 11.65 -Venda: 12.60 13 dias
2017-09-12 - Saldo: 3575.94 Stop: -284.28 | Compra : 12.47 -Venda: 11.55 40 dias
2017-11-17 - Saldo: 3597.64 Stop:  21.70 | Compra : 11.50 -Venda: 11.57 40 dias
2017-12-26 - Saldo: 3294.28 Stop: -303.36 | Compra : 11.35 -Venda: 10.39 21 dias
2018-02-15 - Saldo: 2971.89 Stop: -322.39 | Compra : 10.52 -Venda: 9.49 25 dias
2018-04-03 - Saldo: 2606.90 Stop: -364.99 | Compra : 9.19 -Venda: 8.06 27 dias
2018-05-22 - Saldo: 2379.40 Stop: -227.50 | Compra : 8.00 -Venda: 7.30 24 dias
2018-06-04 - Saldo: 2570.92 Gain:  191.52 | Compra : 7.08 -Venda: 7.65 2 dias
2018-06-22 - Saldo: 2387.01 Stop: -183.9

2016-12-26 - Saldo: 2970.82 Stop: -283.50 | Compra : 51.65 -Venda: 47.15 18 dias
2017-03-06 - Saldo: 2611.14 Stop: -359.68 | Compra : 46.22 -Venda: 40.60 38 dias
2017-03-27 - Saldo: 2450.76 Stop: -160.38 | Compra : 39.52 -Venda: 37.09 10 dias
2017-05-02 - Saldo: 2697.16 Gain:  246.40 | Compra : 37.95 -Venda: 41.80 15 dias
2017-05-26 - Saldo: 2926.56 Gain:  229.40 | Compra : 42.85 -Venda: 46.55 8 dias
2017-07-12 - Saldo: 2584.86 Stop: -341.70 | Compra : 43.32 -Venda: 38.22 29 dias
2017-08-11 - Saldo: 2805.86 Gain:  221.00 | Compra : 37.80 -Venda: 41.05 18 dias
2017-11-17 - Saldo: 2599.07 Stop: -206.79 | Compra : 45.75 -Venda: 42.36 36 dias
2017-12-13 - Saldo: 2354.00 Stop: -245.07 | Compra : 41.19 -Venda: 37.30 15 dias
2018-01-08 - Saldo: 2567.76 Gain:  213.76 | Compra : 36.36 -Venda: 39.70 14 dias
2018-02-22 - Saldo: 2108.08 Stop: -459.68 | Compra : 37.74 -Venda: 30.98 22 dias
2018-03-02 - Saldo: 2288.64 Gain:  180.56 | Compra : 28.40 -Venda: 30.84 5 dias
2018-03-12 - Saldo: 2488.28 Ga

2017-10-02 - Saldo: 3819.20 Gain:  473.20 | Compra : 19.74 -Venda: 22.54 5 dias
2017-10-25 - Saldo: 4175.77 Gain:  356.57 | Compra : 21.00 -Venda: 22.97 8 dias
2017-11-16 - Saldo: 3836.08 Stop: -339.69 | Compra : 20.77 -Venda: 19.08 11 dias
2017-12-08 - Saldo: 3627.53 Stop: -208.55 | Compra : 17.77 -Venda: 16.80 12 dias
2018-01-26 - Saldo: 3923.58 Gain:  296.05 | Compra : 18.97 -Venda: 20.52 11 dias
2018-02-16 - Saldo: 4276.66 Gain:  353.08 | Compra : 20.17 -Venda: 21.99 3 dias
2018-03-15 - Saldo: 4805.73 Gain:  529.07 | Compra : 22.34 -Venda: 25.11 5 dias
2018-04-19 - Saldo: 5480.91 Gain:  675.18 | Compra : 25.77 -Venda: 29.40 17 dias
2018-06-01 - Saldo: 4906.00 Stop: -574.91 | Compra : 28.56 -Venda: 25.55 22 dias
2018-06-11 - Saldo: 5353.20 Gain:  447.20 | Compra : 22.73 -Venda: 24.81 2 dias
2018-07-17 - Saldo: 5817.80 Gain:  464.60 | Compra : 26.41 -Venda: 28.71 4 dias
2018-08-03 - Saldo: 6469.70 Gain:  651.90 | Compra : 28.25 -Venda: 31.43 5 dias
2018-09-10 - Saldo: 5628.22 Stop: -

...

2017-11-21 - Saldo: 6373.65 Gain:  537.00 | Compra : 77.31 -Venda: 84.47 3 dias
2018-02-02 - Saldo: 5984.40 Stop: -389.25 | Compra : 84.80 -Venda: 79.61 40 dias
2018-04-05 - Saldo: 5158.65 Stop: -825.75 | Compra : 79.47 -Venda: 68.46 40 dias
2018-06-05 - Saldo: 5090.89 Stop: -67.76 | Compra : 66.41 -Venda: 65.53 40 dias
2018-07-13 - Saldo: 5613.01 Gain:  522.12 | Compra : 66.38 -Venda: 73.25 14 dias
2018-08-24 - Saldo: 6075.09 Gain:  462.08 | Compra : 73.49 -Venda: 79.57 21 dias
2018-10-25 - Saldo: 5183.09 Stop: -892.00 | Compra : 75.56 -Venda: 64.41 40 dias
2019-01-02 - Saldo: 4666.49 Stop: -516.60 | Compra : 62.46 -Venda: 56.16 40 dias
2019-01-09 - Saldo: 5039.99 Gain:  373.50 | Compra : 56.03 -Venda: 60.53 4 dias
2019-03-08 - Saldo: 5474.59 Gain:  434.60 | Compra : 60.87 -Venda: 66.17 30 dias
2019-05-17 - Saldo: 5302.78 Stop: -171.81 | Compra : 65.38 -Venda: 63.31 40 dias
2019-07-04 - Saldo: 5864.88 Gain:  562.10 | Compra : 72.47 -Venda: 80.17 7 dias
2019-08-01 - Saldo: 6365.13 Gain

2017-02-13 - Saldo: 5329.33 Gain:  431.52 | Compra : 28.03 -Venda: 30.51 10 dias
2017-04-26 - Saldo: 4987.44 Stop: -341.89 | Compra : 29.74 -Venda: 27.83 40 dias
2017-05-16 - Saldo: 5519.20 Gain:  531.76 | Compra : 27.05 -Venda: 29.94 12 dias
2017-06-16 - Saldo: 6131.02 Gain:  611.82 | Compra : 26.68 -Venda: 29.65 20 dias
2017-07-13 - Saldo: 6708.24 Gain:  577.22 | Compra : 28.23 -Venda: 30.89 12 dias
2017-08-29 - Saldo: 6246.03 Stop: -462.21 | Compra : 31.45 -Venda: 29.28 25 dias
2017-11-13 - Saldo: 5703.69 Stop: -542.34 | Compra : 30.16 -Venda: 27.54 37 dias
2018-01-17 - Saldo: 6306.60 Gain:  602.91 | Compra : 30.06 -Venda: 33.25 30 dias
2018-02-27 - Saldo: 6819.24 Gain:  512.64 | Compra : 32.82 -Venda: 35.49 15 dias
2018-05-24 - Saldo: 5163.24 Stop: -1656.00 | Compra : 32.79 -Venda: 24.79 40 dias
2018-06-20 - Saldo: 4848.85 Stop: -314.39 | Compra : 24.38 -Venda: 22.89 16 dias
2018-07-17 - Saldo: 5339.45 Gain:  490.60 | Compra : 21.66 -Venda: 23.86 16 dias
2018-09-26 - Saldo: 5088.57

2018-08-06 - Saldo: 4653.65 Stop:  312.04 | Compra : 16.09 -Venda: 17.25 40 dias
2018-10-03 - Saldo: 5034.53 Gain:  380.88 | Compra : 16.84 -Venda: 18.22 37 dias
2018-12-14 - Saldo: 5526.68 Gain:  492.15 | Compra : 19.68 -Venda: 21.61 14 dias
2019-01-04 - Saldo: 6201.68 Gain:  675.00 | Compra : 20.44 -Venda: 22.94 8 dias
2019-04-05 - Saldo: 6263.55 Stop:  61.87 | Compra : 23.01 -Venda: 23.24 40 dias
2019-05-31 - Saldo: 6792.62 Gain:  529.07 | Compra : 22.61 -Venda: 24.52 33 dias
2019-07-11 - Saldo: 7399.82 Gain:  607.20 | Compra : 24.57 -Venda: 26.77 17 dias
2019-09-10 - Saldo: 7597.85 Stop:  198.03 | Compra : 25.71 -Venda: 26.40 40 dias
2019-11-22 - Saldo: 7623.14 Stop:  25.29 | Compra : 27.01 -Venda: 27.10 40 dias

 TIMP3
2016-02-16 - Saldo: 3331.74 Gain:  331.74 | Compra : 5.15 -Venda: 5.72 3 dias
2016-05-30 - Saldo: 2992.82 Stop: -338.92 | Compra : 7.26 -Venda: 6.52 40 dias
2016-07-11 - Saldo: 3254.07 Gain:  261.25 | Compra : 6.29 -Venda: 6.84 21 dias
2016-10-06 - Saldo: 3280.05 St

2016-09-08 - Saldo: 4726.55 Gain:  494.05 | Compra : 3.51 -Venda: 3.92 12 dias
2016-10-21 - Saldo: 5213.99 Gain:  487.44 | Compra : 3.49 -Venda: 3.85 27 dias
2016-11-23 - Saldo: 5897.39 Gain:  683.40 | Compra : 3.89 -Venda: 4.40 13 dias
2017-01-04 - Saldo: 6467.96 Gain:  570.57 | Compra : 4.03 -Venda: 4.42 24 dias
2017-02-15 - Saldo: 7031.69 Gain:  563.73 | Compra : 4.93 -Venda: 5.36 6 dias
2017-03-15 - Saldo: 6511.09 Stop: -520.60 | Compra : 5.13 -Venda: 4.75 13 dias
2017-04-17 - Saldo: 5875.21 Stop: -635.88 | Compra : 4.30 -Venda: 3.88 20 dias
2017-05-15 - Saldo: 6417.37 Gain:  542.16 | Compra : 3.90 -Venda: 4.26 7 dias
2017-05-23 - Saldo: 6990.81 Gain:  573.44 | Compra : 3.58 -Venda: 3.90 3 dias
2017-06-26 - Saldo: 7697.10 Gain:  706.29 | Compra : 3.86 -Venda: 4.25 17 dias
2017-08-02 - Saldo: 8580.40 Gain:  883.30 | Compra : 4.79 -Venda: 5.34 11 dias
2017-08-22 - Saldo: 9385.36 Gain:  804.96 | Compra : 5.54 -Venda: 6.06 5 dias
2017-10-04 - Saldo: 10289.34 Gain:  903.98 | Compra : 7.

2017-01-02 - Saldo: 9039.07 Gain:  908.58 | Compra : 3.40 -Venda: 3.78 25 dias
2017-01-31 - Saldo: 8158.12 Stop: -880.95 | Compra : 3.59 -Venda: 3.24 14 dias
2017-03-13 - Saldo: 8880.28 Gain:  722.16 | Compra : 3.84 -Venda: 4.18 3 dias
2017-04-12 - Saldo: 9681.08 Gain:  800.80 | Compra : 3.88 -Venda: 4.23 15 dias
2017-06-02 - Saldo: 7778.30 Stop: -1902.78 | Compra : 4.73 -Venda: 3.80 15 dias
2017-06-19 - Saldo: 8475.98 Gain:  697.68 | Compra : 3.79 -Venda: 4.13 5 dias
2017-07-04 - Saldo: 9202.31 Gain:  726.33 | Compra : 3.85 -Venda: 4.18 10 dias
2017-08-28 - Saldo: 10222.81 Gain:  1020.50 | Compra : 5.86 -Venda: 6.51 5 dias
2017-09-29 - Saldo: 11181.58 Gain:  958.77 | Compra : 7.14 -Venda: 7.81 4 dias
2017-12-13 - Saldo: 10973.82 Stop: -207.76 | Compra : 7.53 -Venda: 7.39 40 dias
2018-01-24 - Saldo: 12194.34 Gain:  1220.52 | Compra : 7.55 -Venda: 8.39 13 dias
2018-04-05 - Saldo: 12629.18 Stop:  434.84 | Compra : 7.85 -Venda: 8.13 40 dias
2018-04-20 - Saldo: 13784.06 Gain:  1154.88 | Co

In [576]:
#deixa no formato para saber o resultado - quantidade/financeiro
total_operacoes = {}
lista_operacoes = []
lista_resultados = []
total_resultado = {}
lista_tempo = []
total_tempo = {}
lll = []
for k,v in doc.items():
    for a,b in v.items():
        lista_operacoes.append(b['Qnt Trades'])
        lista_resultados.append(b['Resultado'])
        lll.append(b['Resultado'])
        for temp in b['Periodo Carregando']:
            lista_tempo.append(temp)
  
    total_operacoes[str(k)] = lista_operacoes
    total_resultado[str(k)] = lista_resultados
    total_tempo[str(k)] = lista_tempo
    #reseta listas
    lista_operacoes = []
    lista_resultados = []
    lista_tempo = []


In [577]:
for k,v in total_operacoes.items():
    s = np.sum(v)
    m = np.mean(v)
    std = np.std(v)
    print(f'Resumo trades para pct: {k}')
    print(f'sum: {s:.2f} mean: {m:.2f} desvio padrao: {std:.2f}\n')

Resumo trades para pct: 0.08
sum: 2165.00 mean: 29.26 desvio padrao: 6.05



In [578]:
for k,v in total_resultado.items():
    s = np.sum(v)
    m = np.mean(v)
    std = np.std(v)
    print(f'Resumo trades para pct: {k}')
    print(f'sum: R${s:.2f} mean: R${m:.2f} desvio padrao: R${std:.2f}\n')

Resumo trades para pct: 0.08
sum: R$681335.17 mean: R$9207.23 desvio padrao: R$11199.59



In [579]:
for k,v in total_tempo.items():
    s = np.sum(v)
    m = np.mean(v)
    std = np.std(v)
    print(f'Resumo trades para pct: {k} (volume de 1 ação)')
    print(f'Sum: {s:.2f} dias  | Mean: {m:.2f} dias | Desvio Padrao: {std:.2f} dias\n')

Resumo trades para pct: 0.08 (volume de 1 ação)
Sum: 43920.00 dias  | Mean: 20.29 dias | Desvio Padrao: 12.78 dias



In [580]:
tdn = pd.Series(trade_negativo)
tdp = pd.Series(trade_positivo)

In [581]:
tdn.describe()

count     685.000000
mean     -460.746365
std       449.572577
min     -4455.570000
25%      -560.000000
50%      -331.470000
75%      -218.050000
max        -3.260000
dtype: float64

In [582]:
tdp.describe()

count    1480.000000
mean      523.612453
std       596.199017
min         0.000000
25%       302.517500
50%       408.275000
75%       586.425000
max      9969.990000
dtype: float64

In [583]:
tdp.sum()

774946.4299999999

In [584]:
tdn.sum()

-315611.26

In [585]:
res = (tdp.sum()+tdn.sum())
print(f'{res:.2f}')

459335.17


In [586]:
ini = 222000

In [587]:
print(f'rentabilidade: {res/ini*100:.2f}%')

rentabilidade: 206.91%


In [588]:
74*3000

222000

In [589]:
1272/606

2.099009900990099

In [590]:
813/323

2.51702786377709

In [591]:
# rentabilidade: 134.19%
# 148952.45


In [592]:
1480.000000/685

2.1605839416058394

In [596]:
2165/48

45.104166666666664

2160